# Heroes of Pymoli - a pandas project

In [1]:
#importing dependencies
import pandas as pd
import numpy as np
import os
import json

In [2]:
#Reading the csv file in a dataframe
purchaseData_df = pd.read_json('Resources/purchase_data.json', orient = "records")
#purchaseData_df = pd.read_json('Resources/purchase_data2.json', orient = "records")

In [3]:
#Combining two dataframes
#something to think about later

## Exploratory analysis

In [4]:
#Getting a glance of the data
purchaseData_df.head()

,Age,Gender,Item ID,Item Name,Price,SN
0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46
2,34,Male,174,Primitive Blade,2.46,Assastnya25
3,21,Male,92,Final Critic,1.36,Pheusrical25
4,23,Male,63,Stormfury Mace,1.27,Aela59


In [5]:
#Exploring data - Getting # of rows and data type of each column
purchaseData_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 780 entries, 0 to 779
Data columns (total 6 columns):
Age          780 non-null int64
Gender       780 non-null object
Item ID      780 non-null int64
Item Name    780 non-null object
Price        780 non-null float64
SN           780 non-null object
dtypes: float64(1), int64(2), object(3)
memory usage: 36.6+ KB


In [6]:
#Checking how many rows and columns this data have 
# this info is already inclusive in above df.info()
purchaseData_df.shape

(780, 6)

In [7]:
#Checking missing values
purchaseData_df.count()

Age          780
Gender       780
Item ID      780
Item Name    780
Price        780
SN           780
dtype: int64

In [8]:
#Checking if any column can act as primary key and holds unique values in each row.
#that column will have same # of unique values as total # of rows (78 in this case)
purchaseData_df.nunique()

Age           37
Gender         3
Item ID      183
Item Name    179
Price        152
SN           573
dtype: int64

In [9]:
#Re-checking missing values
purchaseData_df.isnull().sum()

Age          0
Gender       0
Item ID      0
Item Name    0
Price        0
SN           0
dtype: int64

In [10]:
#drop any na/missing values if required

## Data analysis

### Player Count

In [11]:
# Total Number of Players
player_demographics = purchaseData_df.loc[:, ["Age", "Gender","SN"]]
player_demographics = player_demographics.drop_duplicates()
print(type(player_demographics))
num_players = len(player_demographics.index)
player_df = pd.DataFrame({"Total Players" : [num_players]})
player_df

<class 'pandas.core.frame.DataFrame'>


,Total Players
0,573


### Purchasing Analysis (Total)

In [12]:
# Number of Unique Items
# Average Purchase Price
# Total Number of Purchases
# Total Revenue

#print(purchaseData_df.columns)

purchasing_data = purchaseData_df.loc[:, ["SN", "Item ID", "Item Name", "Price"]]
#removing duplicates
purchasing_data = purchasing_data.drop_duplicates()

#running basic calculations
num_unique_items = purchasing_data["Item ID"].nunique()
avg_price = purchasing_data["Price"].mean()
num_purchases = purchasing_data["Price"].count()
total_revenue = purchasing_data["Price"].sum()

#creating the dataframe
purchase_analysis_summary = pd.DataFrame([{'Number of Unique Items': num_unique_items, 'Average Price':avg_price, \
                                           'Number of Purchases':num_purchases, 'Total Revenue': total_revenue}], \
                                         columns=['Number of Unique Items', 'Average Price','Number of Purchases', 'Total Revenue'])

#reformatting -- rounding, comma separating large numbers and adding currency $sign
purchase_analysis_summary = round(purchase_analysis_summary, 2)
purchase_analysis_summary["Number of Unique Items"] = purchase_analysis_summary["Number of Unique Items"].map("{:,}".format)
purchase_analysis_summary["Average Price"] = purchase_analysis_summary["Average Price"].map("${:,.2f}".format)
purchase_analysis_summary["Number of Purchases"] = purchase_analysis_summary["Number of Purchases"].map("{:,}".format)
purchase_analysis_summary["Total Revenue"] = purchase_analysis_summary["Total Revenue"].map("${:,.2f}".format)

purchase_analysis_summary

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$2.93,777,"$2,279.32"


### Gender Demographics

In [13]:
# Percentage and Count of Male Players
# Percentage and Count of Female Players
# Percentage and Count of Other / Non-Disclosed

gender_data = purchaseData_df.loc[:, ["Age", "Gender", "SN"]]
gender_data = gender_data.drop_duplicates()

#performing basic calculations
total_players = gender_data["Gender"].count()
gender_num = gender_data["Gender"].value_counts()
gender_perc = (100*(gender_num)/total_players).map("{:,.2f}%".format)

gender_df = pd.DataFrame({"Percentage of Players": gender_perc, "Total Count": gender_num })
gender_df

,Percentage of Players,Total Count
Male,81.15%,465
Female,17.45%,100
Other / Non-Disclosed,1.40%,8


### Purchasing Analysis by (Gender)

In [14]:
# The below each broken by gender
# Purchase Count
# Average Purchase Price
# Total Purchase Value
# Normalized Totals

purchase_count = purchaseData_df.groupby("Gender")["Price"].count()
purchase_count

average_price = purchaseData_df.groupby("Gender")["Price"].mean().map("${:,.2f}".format)
average_price

total_price = purchaseData_df.groupby("Gender")["Price"].sum().map("${:,.2f}".format)
total_price

normalized_total = total_price #assuming in this case

purchasing_analysis_gender_df = pd.DataFrame({"Purchase Count": purchase_count, "Average Purchase Price":  average_price,\
                                              "Total Purchase Value": total_price, "Normalized Totals": normalized_total})
purchasing_analysis_gender_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
Gender,,,,
Female,136,$2.82,$382.91,$382.91
Male,633,$2.95,"$1,867.68","$1,867.68"
Other / Non-Disclosed,11,$3.25,$35.74,$35.74


### Age Demographics

In [15]:
# The below each broken into bins of 4 years (i.e. <10, 10-14, 15-19, etc.)
age_bins = [0, 9, 14, 19, 24, 29, 34, 39, 99]
group_names = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

player_demographics["Age Ranges"] = pd.cut(player_demographics["Age"], age_bins, labels=group_names)
player_demographics.head()

total_players = player_demographics["Age Ranges"].count()
total_players
num_players_group = player_demographics["Age Ranges"].value_counts()
percent_players = round(100*(num_players_group)/total_players,2)
percent_players

age_demo_df = pd.DataFrame({"Percentage of Players": percent_players, "Total Count": num_players_group})
age_demo_df.sort_index()

,Percentage of Players,Total Count
<10,3.32,19
10-14,4.01,23
15-19,17.45,100
20-24,45.20,259
25-29,15.18,87
30-34,8.20,47
35-39,4.71,27
40+,1.92,11


### Purchasing Analysis (Age)

In [16]:
# Purchase Count
# Average Purchase Price
# Total Purchase Value
# Normalized Totals

purchaseData_age_df = purchaseData_df.copy()
purchaseData_age_df["Age Ranges"] = pd.cut(purchaseData_age_df["Age"], age_bins, labels=group_names)

purchase_count = purchaseData_age_df.groupby("Age Ranges")["Price"].count()
purchase_count

average_price = purchaseData_age_df.groupby("Age Ranges")["Price"].mean().map("${:,.2f}".format)
average_price

total_price = purchaseData_age_df.groupby("Age Ranges")["Price"].sum().map("${:,.2f}".format)
total_price

normalized_total = total_price #assuming in this case

purchasing_analysis_age_df = pd.DataFrame({"Purchase Count": purchase_count, "Average Purchase Price":  average_price,\
                                              "Total Purchase Value": total_price, "Normalized Totals": normalized_total})
purchasing_analysis_age_df.head()

,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
Age Ranges,,,,
<10,28,$2.98,$83.46,$83.46
10-14,35,$2.77,$96.95,$96.95
15-19,133,$2.91,$386.42,$386.42
20-24,336,$2.91,$978.77,$978.77
25-29,125,$2.96,$370.33,$370.33


### Top Spenders

In [17]:
# Identify the the top 5 spenders in the game by total purchase value, then list (in a table):
# SN
# Purchase Count
# Average Purchase Price
# Total Purchase Value

purchase_count = purchaseData_df.groupby(["SN"])["Price"].count()
purchase_count

average_price = purchaseData_df.groupby(["SN"])["Price"].mean()
average_price

total_price = purchaseData_df.groupby(["SN"])["Price"].sum()
total_price

#sorting first bcoz i need the data to stay numeric before sorting
top_spender = pd.DataFrame({"Purchase Count": purchase_count, "Average Purchase Price":  average_price, \
                            "Total Purchase Value": total_price}).sort_values("Total Purchase Value", ascending = False).head()

#formatting the display - decimal point and $sign, etc.
top_spender ["Average Purchase Price"] = top_spender ["Average Purchase Price"].map("${:,.2f}".format)
top_spender ["Total Purchase Value"] = top_spender ["Total Purchase Value"].map("${:,.2f}".format)

top_spender

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Undirrala66,5,$3.41,$17.06
Saedue76,4,$3.39,$13.56
Mindimnya67,4,$3.18,$12.74
Haellysu29,3,$4.24,$12.73
Eoda93,3,$3.86,$11.58


### Most Popular Items

In [18]:
# Identify the 5 most popular items by purchase count, then list (in a table):
# Item ID
# Item Name
# Purchase Count
# Item Price
# Total Purchase Value


purchase_count = purchaseData_df.groupby(["Item ID", "Item Name"])["Price"].count()
purchase_count

average_price = purchaseData_df.groupby(["Item ID", "Item Name"])["Price"].mean() 
average_price

total_price = purchaseData_df.groupby(["Item ID", "Item Name"])["Price"].sum() 
total_price

item_data = pd.DataFrame({"Purchase Count": purchase_count, "Average Purchase Price":  average_price, \
                            "Total Purchase Value": total_price})

#sorting first bcoz i need the data to stay numeric before sorting
most_popular_item = item_data.sort_values("Purchase Count", ascending = False).head()

#formatting the display - decimal point and $sign, etc.
most_popular_item ["Average Purchase Price"] = most_popular_item ["Average Purchase Price"].map("${:,.2f}".format)
most_popular_item ["Total Purchase Value"] = most_popular_item ["Total Purchase Value"].map("${:,.2f}".format)

most_popular_item

,,Purchase Count,Average Purchase Price,Total Purchase Value
Item ID,Item Name,,,
39,"Betrayal, Whisper of Grieving Widows",11,$2.35,$25.85
84,Arcane Gem,11,$2.23,$24.53
31,Trickster,9,$2.07,$18.63
175,Woeful Adamantite Claymore,9,$1.24,$11.16
13,Serenity,9,$1.49,$13.41


### Most Profitable Items

In [19]:
# Identify the 5 most profitable items by total purchase value, then list (in a table):
# Item ID
# Item Name
# Purchase Count
# Item Price
# Total Purchase Value

most_profitable_item =item_data.sort_values("Total Purchase Value", ascending = False).head()

#formatting the display - decimal point and $sign, etc.
most_profitable_item ["Average Purchase Price"] = most_profitable_item ["Average Purchase Price"].map("${:,.2f}".format)
most_profitable_item ["Total Purchase Value"] = most_profitable_item ["Total Purchase Value"].map("${:,.2f}".format)
most_profitable_item

,,Purchase Count,Average Purchase Price,Total Purchase Value
Item ID,Item Name,,,
34,Retribution Axe,9,$4.14,$37.26
115,Spectral Diamond Doomblade,7,$4.25,$29.75
32,Orenmir,6,$4.95,$29.70
103,Singed Scalpel,6,$4.87,$29.22
107,"Splitter, Foe Of Subtlety",8,$3.61,$28.88
